# Multi Servers Queuing Simulation

flow chart explains the process of clients entering multi servers queuing sytem
![image1](image1.png)

In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

class CenterSimulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure_baker=float('inf')      #departure time from baker
        self.t_departure_able=float('inf')      #departure time from able
        self.dep_sum_baker=0                     #Sum of service times by baker
        self.dep_sum_able=0                     #Sum of service times by able
        self.baker_state=0                     #current state of baker
        self.able_state=0                     #current state of able
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_of_departures_baker=0           #number of customers served by baker
        self.num_of_departures_able=0           #number of customers served by able
        self.num_in_system=0
        
    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival,self.t_departure_baker,self.t_departure_able)  
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure_baker and self.t_arrival<self.t_departure_able:
            self.arrival()
        elif self.t_departure_baker<self.t_arrival and self.t_departure_baker<self.t_departure_able:
            self.baker()
        else:
            self.able()
            
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.baker_state==1 and self.able_state==1:
            self.num_in_q+=1
            self.number_in_queue+=1
            self.t_arrival=self.clock+self.gen_int_arr()


        elif self.baker_state==0 and self.able_state==0:
            self.able_state=1
            self.dep2= self.gen_service_time_able()
            self.dep_sum_able += self.dep2
            self.t_departure_able=self.clock + self.dep2
            self.t_arrival=self.clock+self.gen_int_arr()


        elif self.baker_state==0 and self.able_state ==1:       #if able is busy customer goes to Baker
            self.dep1= self.gen_service_time_baker()
            self.dep_sum_baker += self.dep1
            self.t_departure_baker=self.clock + self.dep1
            self.t_arrival=self.clock+self.gen_int_arr()
            self.baker_state=1
        else:                                              #otherwise customer goes to Able
            self.dep2= self.gen_service_time_able()
            self.dep_sum_able += self.dep2
            self.t_departure_able=self.clock + self.dep2
            self.t_arrival=self.clock+self.gen_int_arr()
            self.able_state=1





                
    def baker(self):                #departure from Baker
        self.num_of_departures_baker += 1
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_baker()
            self.dep_sum_baker += self.dep1
            self.t_departure_baker=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure_baker=float('inf') 
            self.baker_state=0                  
    
    def able(self):                #departure from Able
        self.num_of_departures_able += 1
        if self.num_in_q>0:
            self.dep2= self.gen_service_time_able()
            self.dep_sum_able += self.dep2
            self.t_departure_able=self.clock + self.dep2
            self.num_in_q-=1
        else:
            self.t_departure_able=float('inf')
            self.able_state=0
            
            
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    
    def gen_service_time_baker(self):                                #function to generate service time for teller 1 using inverse trnasform (Baker is slower from Able so he takes longer service times)
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 2)
    
    def gen_service_time_able(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.1)
    

s=CenterSimulation()
df=pd.DataFrame(columns=['Average interarrival time','Average service time baker','Average service time Able','Utilization Baker','Utilization Able','People who had to wait in line','Total average wait time'])


for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum_baker/s.num_of_departures_baker,s.dep_sum_able/s.num_of_departures_able,s.dep_sum_baker/s.clock,s.dep_sum_able/s.clock,s.number_in_queue,s.total_wait_time],index=df.columns)
    df=df.append(a,ignore_index=True)   
    


df

,Average interarrival time,Average service time baker,Average service time Able,Utilization Baker,Utilization Able,People who had to wait in line,Total average wait time
0,3.354362,1.715341,0.896484,0.085229,0.222716,3.0,2.286931
1,2.559649,1.803057,1.279163,0.239801,0.329617,14.0,15.453538
2,2.735391,1.060246,0.908162,0.088092,0.256549,2.0,0.063974
3,2.897822,1.584024,1.038088,0.091104,0.294261,4.0,4.380135
4,3.098845,2.634542,1.167375,0.174394,0.294609,8.0,8.421442
...,...,...,...,...,...,...,...
95,3.046365,2.053214,1.183032,0.168497,0.286402,9.0,7.362459
96,2.869316,2.318782,1.255635,0.221274,0.312577,5.0,3.567587
97,2.847879,2.162364,1.431722,0.22332,0.348956,17.0,21.916398
98,3.255123,2.189673,1.012438,0.109084,0.252186,0.0,0.0


# Single Server Queuing Simulation

this flow chart show depicts the code below for single server simulation 
![flowChart2](./image.jpeg)

In [16]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

class CenterSimulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure_baker=float('inf')      #departure time from baker
        self.dep_sum_baker=0                     #Sum of service times by baker
        self.baker_state=0                     #current state of baker
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #customers who had to wait in line(counter)
        self.num_of_departures_baker=0           #number of customers served by baker
        self.num_in_system=0
        
    def time_adv(self):                                                       
        t_next_event=min(self.t_arrival,self.t_departure_baker)  
        self.total_wait_time += (self.num_in_q*(t_next_event-self.clock))
        self.clock=t_next_event
                
        if self.t_arrival<self.t_departure_baker:
            self.arrival()
        else:
            self.baker()

            
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.baker_state==1:                    #if baker is busy people will wait in the queue
            self.num_in_q+=1
            self.number_in_queue+=1
            self.t_arrival=self.clock+self.gen_int_arr()


        else:                                                     # if baker is idle client will get served
            self.dep1= self.gen_service_time_baker()
            self.dep_sum_baker += self.dep1
            self.t_departure_baker=self.clock + self.dep1
            self.t_arrival=self.clock+self.gen_int_arr()
            self.baker_state=1

  
    def baker(self):                #departure from Baker
        self.num_of_departures_baker += 1
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_baker()
            self.dep_sum_baker += self.dep1
            self.t_departure_baker=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure_baker=float('inf') 
            self.baker_state=0                          
            
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    
    def gen_service_time_baker(self):                                #function to generate service time for teller 1 using inverse trnasform (Baker is slower from Able so he takes longer service times)
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.9)
    

s=CenterSimulation()
df1=pd.DataFrame(columns=['Average interarrival time','Average service time baker','Utilization Baker','People who had to wait in line','Total average wait time'])


for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum_baker/s.num_of_departures_baker,s.dep_sum_baker/s.clock,s.number_in_queue,s.total_wait_time],index=df1.columns)
    df1=df1.append(a,ignore_index=True)   
    


df1

,Average interarrival time,Average service time baker,Utilization Baker,People who had to wait in line,Total average wait time
0,2.763047,2.085618,0.746149,60.0,406.421307
1,2.756512,2.004849,0.719049,65.0,291.705469
2,2.613436,1.620619,0.61337,56.0,193.642664
3,3.000098,1.626379,0.535332,34.0,65.900818
4,3.302136,1.889679,0.540903,40.0,147.860943
...,...,...,...,...,...
95,2.739194,2.230209,0.767924,70.0,479.933206
96,2.581032,2.31113,0.885903,81.0,601.892088
97,3.327429,2.100141,0.622514,50.0,264.376443
98,3.060466,1.839961,0.593593,53.0,222.680624


it would be a mess if baker was working alone!